<a href="https://colab.research.google.com/github/indranildchandra/Devanagri_Character_Recognition_Classifier/blob/master/src/Devanagri_Character_Recognition_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Dataset download link -> http://bit.ly/datagst

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#upload data.zip file for training

In [0]:
!unzip data.zip

In [0]:
!ls -lrt